### Входные данные
Форма для сдачи: https://goo.gl/forms/qptFQ1ukuCxeMEA63

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
#####Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
#####Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 3 знака после запятой.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [1]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in xrange(10000000):
        easy += random_generator.randint(0, 2) # Неубывает
        medium = random_generator.randint(0, 256 - 1) # Можно уложить в uint8
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [2]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return [round(x, 3) for x in result / count]

In [3]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print get_tuple_stream_mean(example(easy_stream()), 2)

[4998568.825, 4998578.825]
Wall time: 2min 18s


In [4]:
from collections import deque
def movmean(stream, N):
    assert N == int(N) and N > 0, "N must be an integer >0"
    fN = float(N)
    sum1 = 0
    sum2 = 0
    queue=deque([(0, 0)]*N)
    for val in stream:
        sum1 += val
        sum2 += val*val
        queue.append((sum1, sum2))
        s1, s2 = queue.popleft()
        mean = (sum1-s1)/fN
        variance = (sum2-s2)/fN-mean**2
        yield mean, variance

In [5]:
# Test
for i in movmean([x for x in range(10)], 2):
    print i
_list = [x for x in range(10)]
for i,val in enumerate(_list):
    if i>=1:
        print (_list[i]+_list[i-1])/2., (_list[i]**2+_list[i-1]**2)/2.0-((_list[i]+_list[i-1])/2.0)**2
    else:
        print 0.0, 0.0

(0.0, 0.0)
(0.5, 0.25)
(1.5, 0.25)
(2.5, 0.25)
(3.5, 0.25)
(4.5, 0.25)
(5.5, 0.25)
(6.5, 0.25)
(7.5, 0.25)
(8.5, 0.25)
0.0 0.0
0.5 0.25
1.5 0.25
2.5 0.25
3.5 0.25
4.5 0.25
5.5 0.25
6.5 0.25
7.5 0.25
8.5 0.25


In [6]:
def easyminmaxmed(stream, N):
    assert N == int(N) and N > 0, "N must be an integer >0"
    fN = float(N)
    queue=deque([0]*(N-1))
    for val in stream:
        
        queue.append(val)
        max_ = val
        if N%2 == 0:
            n = N/2
            med_ = (queue[n-1]+queue[n])/2.0
        else:
            med_ = float(queue[N/2])    
        min_ = queue.popleft()
        yield min_, med_, max_

In [7]:
# Test
for i in easyminmaxmed([x for x in range(10)], 4):
    print i
_list = [x for x in range(10)]
for i,val in enumerate(_list):
    if i>=2:
        print min(_list[i-2:i+1]),  np.median(_list[i-2:i+1]), max(_list[i-2:i+1])
    else:
        print "Tututut"

(0, 0.0, 0)
(0, 0.0, 1)
(0, 0.5, 2)
(0, 1.5, 3)
(1, 2.5, 4)
(2, 3.5, 5)
(3, 4.5, 6)
(4, 5.5, 7)
(5, 6.5, 8)
(6, 7.5, 9)
Tututut
Tututut
0 1.0 2
1 2.0 3
2 3.0 4
3 4.0 5
4 5.0 6
5 6.0 7
6 7.0 8
7 8.0 9


In [8]:
import bisect
def mediumminmaxmed(stream, N):
    assert N == int(N) and N > 0, "N must be an integer >0"
    sorted_list = [0]*(N-1)
    queue = deque(sorted_list)
    for val in stream:
        # Вставляем элемент в очередь и в отсортированный список
        bisect.insort(sorted_list, val)
        queue.append(val)
        # Максимум в правом конце отсортированного списка
        max_ = sorted_list[-1]
        # медиана в середине
        if N%2 == 0:
            n = N/2
            med_ = (sorted_list[n-1]+sorted_list[n])/2.0
        else:
            med_ = float(sorted_list[N/2]) 
        # Минимум в левом конце
        min_ = sorted_list[0]
        sorted_list.remove(queue.popleft())
        yield min_, med_, max_

In [9]:
# Test
for i in mediumminmaxmed([1,3,5,4,2,7,9,8,6,0], 3):
    print i
_list = [1,3,5,4,2,7,9,8,6,0]
for i,val in enumerate(_list):
    if i>=2:
        print min(_list[i-2:i+1]),  np.median(_list[i-2:i+1]), max(_list[i-2:i+1])
    else:
        print "Tututut"

(0, 0.0, 1)
(0, 1.0, 3)
(1, 3.0, 5)
(3, 4.0, 5)
(2, 4.0, 5)
(2, 4.0, 7)
(2, 7.0, 9)
(7, 8.0, 9)
(6, 8.0, 9)
(0, 6.0, 8)
Tututut
Tututut
1 3.0 5
3 4.0 5
2 4.0 5
2 4.0 7
2 7.0 9
7 8.0 9
6 8.0 9
0 6.0 8


In [10]:
%%time
print "Mean and variance for different streams"
easy_tuple = get_tuple_stream_mean(movmean(easy_stream(), 1000), 2)
print "Easy: ", easy_tuple
medium_tuple = get_tuple_stream_mean(movmean(medium_stream(), 1000), 2)
print "Medium: ", medium_tuple
nightmare_tuple = get_tuple_stream_mean(movmean(nightmare_stream(), 1000), 2)
print "Nightmare: ", nightmare_tuple

Mean and variance for different streams
Easy:  [4998069.5, 83384.969]
Medium:  [127.487, 5452.51]
Nightmare:  [499884857.662, 8.322319064793309e+16]
Wall time: 8min 1s


In [11]:
%%time
easy_mmm = get_tuple_stream_mean(easyminmaxmed(easy_stream(), 1000), 3)
print "easy_stream min med max", easy_mmm

easy_stream min med max [4997570.192, 4998069.496, 4998568.825]
Wall time: 2min 24s


In [12]:
%%time
medium_mmm = get_tuple_stream_mean(mediumminmaxmed(medium_stream(), 1000), 3)
print "medium_stream min med max", medium_mmm 

medium_stream min med max [0.02, 127.514, 254.979]
Wall time: 4min 27s


In [13]:
%%time
nightmare_mmm = get_tuple_stream_mean(mediumminmaxmed(nightmare_stream(), 1000), 3)
print "nightmare_stream min med max", nightmare_mmm

nightmare_stream min med max [1000380.616, 499779897.74, 999007588.072]
Wall time: 4min 34s
